In [ ]:
from ipywidgets import interact

import math

def calc_x_reserves(vault_apy,y_reserves,tranche_length,time_stretch):
  t=tranche_length/(365*time_stretch)
  T=tranche_length/365
  return y_reserves/((1-T*(vault_apy/100))**(-1/t)-1)

def calc_price(x_reserves,y_reserves,total_supply,days_until_maturity,time_stretch):
  t=days_until_maturity/(365*time_stretch)
  return 1/((y_reserves + total_supply)/x_reserves)**t

def calc_k(in_reserves,out_reserves,t):
  return pow(in_reserves,1-t) + pow(out_reserves,1-t)

def calc_apy(price,days_until_maturity):
  T=days_until_maturity/365
  return (1-price)/T * 100

def is_trade_valid(in_,in_reserves,out_reserves,t):
  k = pow(in_reserves,1-t) + pow(out_reserves,1-t)
  check = math.log(k)/math.log(in_reserves + in_) + t
  return True if check >= 1 else False

def calc_max_trade(in_reserves,out_reserves,t):
  k = pow(in_reserves,1-t) + pow(out_reserves,1-t)
  return k**(1/(1-t)) - in_reserves

def calc_out_given_in(in_,in_reserves,out_reserves,token_out,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = out_reserves - pow(k-pow(in_reserves+in_,1-t),1/(1-t))
        if token_out == "base":
            fee =  (in_-without_fee)*g
            with_fee = without_fee-fee
        elif token_out == "fyt":
            fee =  (without_fee-in_)*g
            with_fee = without_fee-fee   
        return (with_fee,without_fee,fee)
    
    
def calc_max_percent_apy_change(vault_apy,y_reserves,tranche_length,time_stretch):
    g = 0.1
    epsilon = .0000001
    x_reserves = calc_x_reserves(vault_apy,y_reserves,tranche_length,time_stretch)
    print("Required Base Reserves: " + str(x_reserves))
    total_supply = x_reserves
    t = tranche_length/(365*time_stretch)
    max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t) - epsilon
    print("Max Valid Trade: {:}".format(max_trade))
    (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t)
    unit_price = without_fee/max_trade
    print("FYT Unit Price: {:}".format(unit_price))
    resulting_apy=calc_apy(unit_price,tranche_length)
    print("Resulting APY: {:}".format(resulting_apy))
    max_percent_apy_change = (resulting_apy-vault_apy)/vault_apy * 100
    return 'Max Percent APY Change: {}'.format(max_percent_apy_change)

In [28]:
interact(calc_max_percent_apy_change,vault_apy=(1,50,1),y_reserves=(50,1000,50),tranche_length=(30,360,30),time_stretch=(5,50,5));

interactive(children=(IntSlider(value=25, description='vault_apy', max=50, min=1), IntSlider(value=500, descri…

In [103]:
import nbinteract as nbi
import numpy as np

def x_values(): return np.arange(1, 50)
def y_values(xs,y_reserves,tranche_length,time_stretch):
    g = 0.1
    epsilon = .0000001
    resulting_apys = []
    max_percent_apy_changes = []
    for vault_apy in xs:
        x_reserves = calc_x_reserves(vault_apy,y_reserves,tranche_length,time_stretch)
        total_supply = x_reserves
        t = tranche_length/(365*time_stretch)
        max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t) - epsilon
        (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t)
        unit_price = without_fee/max_trade
        resulting_apy=calc_apy(unit_price,tranche_length)
        resulting_apys.append(resulting_apy)
        max_percent_apy_change = (resulting_apy-vault_apy)/vault_apy * 100
        max_percent_apy_changes.append(max_percent_apy_change)
    return np.array(max_percent_apy_changes)


def setOptions(xlim,ylim):
    global opts
    opts = {
        'xlim': (0, xlim),
        'ylim': (0, ylim),
        'xlabel': 'Vault APY',
        'ylabel': 'Max Percent APY Change',
        'title': 'Vault APY vs Max Percent APY Change',
        'animation_duration': 100,
    }
    
foo=interact(setOptions,xlim=(0,50),ylim=(0,3000))


interactive(children=(IntSlider(value=25, description='xlim', max=50), IntSlider(value=1500, description='ylim…

In [106]:
nbi.line(x_values, y_values, y_reserves=(50,1000,50),tranche_length=(30,360,30),time_stretch=(5,50,5), options=opts)

In [107]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...